In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as func
from scipy.io import loadmat
import numpy as np
from sklearn.feature_extraction import image
import pandas as pd
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import gensim

In [2]:
import re
import pickle
import json
import xml.etree.ElementTree as ET
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import os
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
import math
from difflib import SequenceMatcher
import pandas as pd

In [3]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
year_regex = re.compile(r'((19[0-9]{2})|(20[0-9]{2}))[a-z]?')
conversion_dict = {}
stop_words = [',', '.', '(', ')', ':', '-', "+", ";", "a", "about", "al", "al.", "all", 
	"already", "also", "although", "am", "an", "and", "another", "any", "anyhow", "are", 
	"aren", "aren't", "around", "as", "at", "back", "be", "because", "been", 
	"being", "beyond", "but", "by", "can", "cannot", "cant", "co", "con", "could", "couldn", 
	"couldnt", "d", "de", "did", "didn", "didn't", "do", "does", "doesn", "doesn't", 
	"doing", "don", "don't", "done", "due", "each", "either", "else", "elsewhere", "et", 
	"etc", "even", "ever", "except", "for", "found", "from", "further", "had", "hadn", 
	"hadn't", "has", "hasn", "hasn't", "hasnt", "have", "haven", "haven't", "having", 
	"he", "hence", "her", "here", "hereafter", "hereby", "hers", 
	"herself", "him", "himself", "his", "how", "however", "i", "ie", "if", "in", "inc", 
	"indeed", "interest", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", 
	"just", "ltd", "ll", "m", "may", "me", "meanwhile", "might", "mightn", 
	"mightn't", "mine", "moreover", "most", "mostly", "move", "much", "must", "mustn", 
	"mustn't", "my", "myself", "name", "namely", "need", "needn", "needn't", "neither", 
	"nevertheless", "no", "nobody", "noone", "nor", "not", "now", "nowhere", "o", "of", 
	"off", "often", "on", "only", "onto", "or", "other", "others", "otherwise", "our", "own", 
	"per", "perhaps", "put", "rather", "re", "s", "same", "see", "seem", "seemed", 
	"seeming", "seems", "serious", "she", "should", "shouldn", "shouldn't", "since", 
	"sincere", "so", "some", "somehow", "someone", "something", "somewhere", "still", 
	"such", "t", "take", "than", "that", "that'll", "the", "their", "theirs", 
	"them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", 
	"therein", "thereupon", "these", "they", "this", "those", "though", "throughout", 
	"thru", "thus", "to", "together", "too", "toward", "towards", "un", "until", "upon", 
	"us", "ve", "very", "via", "was", "wasn", "wasn't", "we", "well", "were", "weren", 
	"weren't", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", 
	"whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", 
	"whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", 
	"without", "won", "won't", "would", "wouldn", "wouldn't", "y", "yet", "you", "your", 
	"yours", "yourself", "yourselves", "from SVM import SVCone", "two", "three", "four", "five", "six", "seven",
	"eight", "nine", "zero", "between", 'below', 'ourselves', "you'll", 'again', 'once', 'over', 'shan', 'few', 
    'against', 'before', 'out', 'down', 'both', 'up', "you've", "shan't", "you're", "should've", 'ours', 'ma', 
    "couldn't", 'during', 'more', 'ain', 'through', 'after', 'above', "she's", "you'd", 'under' ]

In [4]:
def get_citations(folder):
    citation_list = {}
    for file in os.listdir(folder) :
        tree = ET.parse(folder+file)
        root = tree.getroot()
        id = file[:8]
        for element in root.iterfind("algorithm"):
            if(element.attrib['name']=="ParsCit"):
                citlist = element.getchildren()
                cits = citlist[0].getchildren()
                citations = []
                for cit in cits:
                    cit_dict = {}
                    if(cit.attrib['valid']=="true"):
                        try :
                            title = cit.find('title').text.lower()
                        except :
                            title = cit.find('rawString').text.lower()
                        cit_dict['title'] = title
                        cit_dict['cit'] = cit
                        citations.append(cit_dict)
                
                citation_list[id] = citations
            
    return citation_list 

In [5]:
citations = get_citations("../xmls/")

In [6]:
def get_words(string, cit_auths=None):

    string = string.replace('-', '').lower()
    context_words = word_tokenize(string)
    final_context_words = []

    for word in context_words:
        if word in stop_words:
            continue
        if re.fullmatch(year_regex, word): #or match_auths(word, cit_auths):
            continue
        if re.fullmatch(r'[0-9]+([.][0-9]+)?', word):
            word = '<number>'
            final_context_words.append(word)
        elif 'this_citation' in word:
            final_context_words.append('<this_citation>')
        elif re.fullmatch(r'[a-z]+', word):
            original_word = word
            final_word = lemmatizer.lemmatize(word)
            
            if final_word not in conversion_dict:
                conversion_dict[final_word] = {}

            if original_word not in conversion_dict[final_word]:
                conversion_dict[final_word][original_word] = 0
            conversion_dict[final_word][original_word]+=1

            final_context_words.append(final_word)

    return final_context_words

In [7]:
def get_contexts(citations) :
    dataset = {}
    count = 0
    for key in citations.keys():
        context_list = []
        for cit in citations[key] :
            dict1 = {}
            dict1['paper_name'] = cit['title']
            dict1['context'] = ""
            for context in cit['cit'].findall('contexts/context') :
                text = context.text.lower()
                citstr = context.get('citStr').lower()
                text = text.replace(citstr, "this_citation")
                dict1['context']+= text+" "
            dict1['context'] = get_words(dict1['context'])
            context_list.append(dict1)
        dataset[key] = context_list
        count+=1
        if(count%100==0) :
            print(count)
    return dataset

In [8]:
contexts = get_contexts(citations)

100
200
300
400
500
600
700
800
900
1000
1100


In [9]:
embeddings_dict = {}
with open("glove.6B.100d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [10]:
mapping = {}
mapping_rev = {}
values = []
count = 0
for key in contexts :
    papers = contexts[key]
    for pap in papers :
        name = pap['paper_name']
        words = pap['context']
        if(len(words)>0) :
            mapping[key+"_|_"+name] = count
            mapping_rev[count] = key+"_|_"+name
            count+=1
            embed = []
            if(len(words)>250) :
                words = words[:250]
            for word in words :
                if(word in embeddings_dict) :
                    embed.append(embeddings_dict[word])
                else :
                    embed.append(embeddings_dict['unk'])
            diff = 250-len(words)
            if(diff>0) :
                for i in range(diff) :
                    embed.append(embeddings_dict['unk'])
            values.append(embed)  

In [11]:
print(len(values))

27635


In [12]:
tags = pickle.load(open("../pickles_data/baseline_tags.pkl","rb"))

In [13]:
output = []
for i in range(count) :
    key,name = mapping_rev[i].split("_|_")
    papers = tags[key]
    for pap in papers :
        if(pap['paper_name']==name) :
            output.append(pap['tag'])
            break

In [14]:
print(len(output))

27635


In [15]:
citation_worthy = pickle.load(open("base_pickles/citation_worthy.pkl", "rb"))
section_contexts = pickle.load(open("base_pickles/section_contexts.pkl", "rb"))

In [16]:
context_values = []
context_out = []
count1 = 0
max1 = 0
map_val = {'experiment':0, 'conclusion':1, 'other_sections':2, 'related_work':3, 'method':4, 'introduction':5, 'evaluation':6}
for context in section_contexts :
    words = get_words(context)
    if(len(words)>max1) :
        max1 = len(words)
    embed = []
    for word in words :
        if(word in embeddings_dict) :
            embed.append(embeddings_dict[word])
        else :
            embed.append(embeddings_dict['unk'])
    diff = 250-len(words)
    if(diff>0) :
        for i in range(diff) :
            embed.append(embeddings_dict['unk'])
    context_values.append(embed)  
    context_out.append(map_val[section_contexts[context]])
    count1+=1
    if(count1%5000==0) :
        print(count1)
print(max1)

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
205


In [17]:
worthy_values = []
worthy_out = []
count2 = 0
max1 = 0
all_zeros = []
all_ones = []
all_data = []
for line in citation_worthy :
    if(citation_worthy[line]==1) :
        all_ones.append(line)
    else :
        all_zeros.append(line)
#     words = get_words(line)
#     if(len(words)>max1) :
#         max1 = len(words)

ar = np.random.choice(len(all_zeros), len(all_ones))
for x in ar :
    all_data.append(all_zeros[x])
    
for y in all_ones :
    all_data.append(y)
    
avg = 0
for line in all_data :
    words = get_words(line)
    embed = []
    if(len(words)>250) :
        for i in range(250) :
            word = words[i]
            if(word in embeddings_dict) :
                embed.append(embeddings_dict[word])
            else :
                embed.append(embeddings_dict['unk'])
    else :
        for word in words :
            if(word in embeddings_dict) :
                embed.append(embeddings_dict[word])
            else :
                embed.append(embeddings_dict['unk'])
        diff = 250 - len(words)
        if(diff>0) :
            for i in range(diff) :
                embed.append(embeddings_dict['unk'])
    worthy_values.append(embed)  
    worthy_out.append(citation_worthy[line])

In [18]:
size = len(values)
trainset = values[:int(0.8*size)]
trainlabel = output[:int(0.8*size)]
testset = values[int(0.8*size):]
testlabel = output[int(0.8*size):]

In [19]:
main_vals = np.random.choice(len(trainset), len(worthy_values)-len(trainset))
section_vals = np.random.choice(len(context_values),len(worthy_values)-len(context_values))

In [20]:
for x in main_vals :
    trainset.append(trainset[x])
    trainlabel.append(trainlabel[x])

In [21]:
for y in section_vals :
    context_values.append(context_values[y])
    context_out.append(context_out[y])

In [22]:
print(len(trainset))
print(len(trainlabel))
print(len(context_values))
print(len(context_out))
print(len(worthy_values))
print(len(worthy_out))

90868
90868
90868
90868
90868
90868


In [ ]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import CuDNNLSTM, Bidirectional, Dropout

def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


def model_lstm_atten(input1):
    inp = input1
    x = Bidirectional(CuDNNLSTM(50, return_sequences=True))(inp)
    x = AttentionWithContext()(x)
    x = Dense(32, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [23]:
class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)
        
        self.supports_masking = True

        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0
        
        weight = torch.zeros(feature_dim, 1)
        nn.init.kaiming_uniform_(weight)
        self.weight = nn.Parameter(weight)
        
        if bias:
            self.b = nn.Parameter(torch.zeros(step_dim))
        
    def forward(self, x, mask=None):
        feature_dim = self.feature_dim 
        step_dim = self.step_dim

        eij = torch.mm(
            x.contiguous().view(-1, feature_dim), 
            self.weight
        ).view(-1, step_dim)
        
        if self.bias:
            eij = eij + self.b

#         print("eij",eij.shape)
#         print("x",x.contiguous().view(-1,feature_dim).shape)
            
        eij = torch.tanh(eij)
        a = torch.exp(eij)
        
        if mask is not None:
            a = a * mask

        a = a / (torch.sum(a, 1, keepdim=True) + 1e-10)

        weighted_input = x * torch.unsqueeze(a, -1)
        return torch.sum(weighted_input, 1)

class Attention_Net(nn.Module):
    def __init__(self):
        super(Attention_Net, self).__init__()
        drp = 0.2
        embed_size = 100
        # self.embedding = nn.Embedding(max_features, embed_size)
        # # self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        # self.embedding.weight.requires_grad = False

        # self.embedding_dropout = nn.Dropout2d(0.1)
        self.lstm = nn.LSTM(embed_size, 50, bidirectional=True, batch_first=True)
#         self.lstm2 = nn.LSTM(64*2, 32, bidirectional=True, batch_first=True)

        self.attention_layer = Attention(100, 250)
        
        self.main_linear = nn.Linear(100 , 32)
        self.relu = nn.ReLU()
        self.main_out = nn.Linear(32, 2)
        self.section_linear = nn.Linear(100 , 20)
        self.section_out = nn.Linear(20, 7)
        self.worthy_linear = nn.Linear(100 , 20)
        self.worthy_out = nn.Linear(20, 2)
        self.main_dropout = nn.Dropout2d(0.2)
        self.worthy_dropout = nn.Dropout2d(0.2)
        self.section_dropout = nn.Dropout2d(0.2)

    def forward(self, x, task_no):
        if(task_no==0) :
            h_embedding = x
            h_lstm, _ = self.lstm(h_embedding)
            h_lstm_atten = self.attention_layer(h_lstm)
            conc = self.relu(self.main_linear(h_lstm_atten))
            conc = self.main_dropout(conc)
            out = self.main_out(conc)
            return out
        elif(task_no==1) :
            h_embedding = x
            h_lstm, _ = self.lstm(h_embedding)
            h_lstm_atten = self.attention_layer(h_lstm)
            conc = self.relu(self.section_linear(h_lstm_atten))
            conc = self.section_dropout(conc)
            out = self.section_out(conc)
            return out   
        else :
            h_embedding = x
            h_lstm, _ = self.lstm(h_embedding)
            h_lstm_atten = self.attention_layer(h_lstm)
            conc = self.relu(self.worthy_linear(h_lstm_atten))
            conc = self.worthy_dropout(conc)
            out = self.worthy_out(conc)
            return out             

In [24]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 128

train_data = TensorDataset(torch.as_tensor(trainset), torch.as_tensor(trainlabel), torch.as_tensor(context_values), torch.as_tensor(context_out),torch.as_tensor(worthy_values), torch.as_tensor(worthy_out))
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(torch.as_tensor(testset), torch.as_tensor(testlabel))
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=128)

In [25]:
from sklearn.metrics import classification_report

def eval_model(model, val_loader) :
    final_out = []
    final_lab = []

    for idx, (val_input, val_label) in enumerate(val_loader):
        val_input = val_input.type(torch.FloatTensor)
        val_input = val_input.cuda()
        output = model(val_input,0)
        output = output.cpu().detach().numpy()
        val_label = val_label.cpu().detach().numpy()
        ar= []
        for i in range(output.shape[0]) :
            index = -1
            max_val = -1
            for j in range(len(output[i])) :
                if(output[i][j]>max_val) :
                    index = j
                    max_val = output[i][j]
            ar.append(index)
    
        val_label = list(val_label)
        final_out.extend(ar)
        final_lab.extend(val_label)

        
    return classification_report(final_lab, final_out)

In [ ]:
learning_rate = 0.01
num_epochs = 25
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
model = Attention_Net().to(device)
weights = [1.0, 5.0]
class_weights = torch.FloatTensor(weights).cuda()
criterion1 = nn.CrossEntropyLoss()
criterion2 = nn.CrossEntropyLoss()
criterion3 = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for batch_id, (train_input, train_label, con_val, con_label, worthy_val, worthy_label) in enumerate(train_dataloader):
        optimizer.zero_grad() 
        train_input = train_input.type(torch.FloatTensor)
        train_input = train_input.cuda()
        output1 = model(train_input,0)
        train_label = train_label.type(torch.LongTensor)
        train_label = train_label.cuda()
        loss1 = criterion1(output1, train_label)
        
        con_val = con_val.type(torch.FloatTensor)
        con_val = con_val.cuda()
        output2 = model(con_val,1)
        con_label = con_label.type(torch.LongTensor)
        con_label = con_label.cuda()
        loss2 = criterion2(output2, con_label)
        
        worthy_val = worthy_val.type(torch.FloatTensor)
        worthy_val = worthy_val.cuda()
        output3 = model(worthy_val,2)
        worthy_label = train_label.type(torch.LongTensor)
        worthy_label = train_label.cuda()
        loss3 = criterion3(output3, worthy_label)
        
        loss = loss1 + 0.1*loss2 + 0.0*loss3
        loss.backward()
        optimizer.step()
        
        if batch_id % 100 == 0:
            print('Epoch [{}/{}], Loss:{:.4f}'
                .format(epoch+1, num_epochs, loss.data))
            print("classification_report")
            print(eval_model(model,val_dataloader))
            print("--------------------------------------------------------------")
            # losses.append(loss.data)
            
        del train_input
        del train_label
        del con_val
        del con_label
        del worthy_val
        del worthy_label
        del output1
        del output2
        del output3
        del loss1
        del loss2
        del loss3
        del loss
        torch.cuda.empty_cache()

Epoch [1/25], Loss:0.8414
classification_report
              precision    recall  f1-score   support

           0       0.90      1.00      0.95      5000
           1       0.00      0.00      0.00       527

    accuracy                           0.90      5527
   macro avg       0.45      0.50      0.47      5527
weighted avg       0.82      0.90      0.86      5527

--------------------------------------------------------------
Epoch [1/25], Loss:0.4060
classification_report
              precision    recall  f1-score   support

           0       0.90      1.00      0.95      5000
           1       0.00      0.00      0.00       527

    accuracy                           0.90      5527
   macro avg       0.45      0.50      0.47      5527
weighted avg       0.82      0.90      0.86      5527

--------------------------------------------------------------
Epoch [1/25], Loss:0.3360
classification_report
              precision    recall  f1-score   support

           0       0.

Epoch [3/25], Loss:0.2570
classification_report
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      5000
           1       0.54      0.21      0.30       527

    accuracy                           0.91      5527
   macro avg       0.73      0.60      0.63      5527
weighted avg       0.89      0.91      0.89      5527

--------------------------------------------------------------
Epoch [3/25], Loss:0.2264
classification_report
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      5000
           1       0.44      0.43      0.44       527

    accuracy                           0.89      5527
   macro avg       0.69      0.69      0.69      5527
weighted avg       0.89      0.89      0.89      5527

--------------------------------------------------------------
Epoch [3/25], Loss:0.1878
classification_report
              precision    recall  f1-score   support

           0       0.

Epoch [5/25], Loss:0.1474
classification_report
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      5000
           1       0.44      0.44      0.44       527

    accuracy                           0.89      5527
   macro avg       0.69      0.69      0.69      5527
weighted avg       0.89      0.89      0.89      5527

--------------------------------------------------------------
